### Preprocess JD data

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [30]:
order = pd.read_csv("./JD_data/JD_order_data.csv")
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549989 entries, 0 to 549988
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_ID                    549989 non-null  object 
 1   user_ID                     549989 non-null  object 
 2   sku_ID                      549989 non-null  object 
 3   order_date                  549989 non-null  object 
 4   order_time                  549989 non-null  object 
 5   quantity                    549989 non-null  int64  
 6   type                        549989 non-null  int64  
 7   promise                     549989 non-null  object 
 8   original_unit_price         549989 non-null  float64
 9   final_unit_price            549989 non-null  float64
 10  direct_discount_per_unit    549989 non-null  float64
 11  quantity_discount_per_unit  549989 non-null  float64
 12  bundle_discount_per_unit    549989 non-null  float64
 13  coupon_discoun

In [31]:
# drop duplicate
order = order.drop_duplicates(subset=['order_ID', 'sku_ID'], keep='first')
# 1P
order_1p = order[order['type'] == 1]
# timestamp --- datetime
order_1p['order_date'] = pd.to_datetime(order_1p['order_date'])
order_1p['order_time'] = pd.to_datetime(order_1p['order_time'])
# order time
order_1p.sort_values(by=['order_time'], ascending=True, inplace=True)
# save
order_1p.to_csv('./pre_data/order_1p.csv', index=False)

C:\Users\18888\AppData\Local\Temp\ipykernel_8304\2457394310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_1p['order_date'] = pd.to_datetime(order_1p['order_date'])
C:\Users\18888\AppData\Local\Temp\ipykernel_8304\2457394310.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_1p['order_time'] = pd.to_datetime(order_1p['order_time'])
C:\Users\18888\AppData\Local\Temp\ipykernel_8304\2457394310.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

In [32]:
order_1p.head(n=20)

,order_ID,user_ID,sku_ID,order_date,order_time,quantity,type,promise,original_unit_price,final_unit_price,direct_discount_per_unit,quantity_discount_per_unit,bundle_discount_per_unit,coupon_discount_per_unit,gift_item,dc_ori,dc_des
18389,ec43a8d555,8095774018,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,10,10
19682,0e8271caa5,69b1f57091,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,9,9
19685,ad2c7ef07b,dd1adaddd6,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,9,9
19709,3de7e67c09,3f5ed6bcb2,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,9,9
6836,03cfac9a9d,58f640c4e5,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,2,162.9,9.9,153.0,0.0,0.0,0.0,0,26,26
30404,c994c2b7a3,6a5c089f3d,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,4,4
3703,2645bb5943,db2a95aed5,416aed740e,2018-03-01,2018-03-01 00:00:00,1,1,1,162.9,9.9,153.0,0.0,0.0,0.0,0,26,26
5900,2f6dd45551,be4b2d6087,416aed740e,2018-03-01,2018-03-01 00:00:01,1,1,2,162.9,9.9,153.0,0.0,0.0,0.0,0,24,24
34187,77b7d4ccca,54b9524fe2,416aed740e,2018-03-01,2018-03-01 00:00:01,1,1,2,162.9,9.9,153.0,0.0,0.0,0.0,0,7,53
34189,1675ba953d,a3d4831849,416aed740e,2018-03-01,2018-03-01 00:00:02,1,1,2,162.9,9.9,153.0,0.0,0.0,0.0,0,7,53


### Orders fulfilled by FDC and its RDC

In [33]:
network = pd.read_csv("./JD_data/JD_network_data.csv")
order = pd.read_csv("./pre_data/order_1p.csv")

In [43]:
# RDC
rdc_list = fdc_list = network['region_ID'].unique().tolist()
print(rdc_list)
# RDC orders
rdc_order_num = {}
for rdc in rdc_list:
    tmp_rdc_order = order[order['dc_des'] == rdc]
    rdc_order_num[rdc] = len(tmp_rdc_order['order_ID'].unique().tolist())
print(rdc_order_num)

[2, 3, 4, 5, 7, 9, 10, 24]
{2: 21842, 3: 1274, 4: 18197, 5: 29222, 7: 7279, 9: 20637, 10: 5131, 24: 10944}


In [44]:
# FDC
fdc_list = network['dc_ID'].unique().tolist()
print(fdc_list)
# FDC orders
dc_order = pd.DataFrame(columns=['DC', 'RDC', 'total_order_num', 'order_by_DC', 'order_by_RDC', 'sum_DC_RDC'], dtype='Int64')
for fdc in fdc_list:
    rdc = int(network[network['dc_ID'] == fdc]['region_ID'])
    tmp = []
    tmp_fdc_order = order[order['dc_des'] == fdc]
    tmp1 = tmp_fdc_order[tmp_fdc_order['dc_ori'] == fdc]['order_ID'].unique().tolist()
    tmp2 = tmp_fdc_order[tmp_fdc_order['dc_ori'] == rdc]['order_ID'].unique().tolist()
    if fdc == rdc:
        tmp = [fdc, rdc, rdc_order_num[fdc], len(tmp1), 0, len(tmp1)]
    else:
        tmp = [fdc, rdc, len(tmp_fdc_order), len(tmp1), len(tmp2), len(tmp1) + len(tmp2)]
    dc_order.loc[len(dc_order.index)] = tmp
print(dc_order)


[57, 43, 42, 66, 20, 58, 15, 6, 2, 64, 14, 33, 34, 35, 3, 45, 44, 51, 63, 19, 4, 12, 28, 5, 13, 55, 54, 26, 65, 53, 39, 8, 46, 31, 67, 1, 7, 59, 9, 56, 25, 27, 32, 61, 36, 50, 37, 11, 10, 38, 47, 40, 41, 52, 24, 21]
    DC  RDC  total_order_num  order_by_DC  order_by_RDC  sum_DC_RDC
0   57    2                0            0             0           0
1   43    2             4192         2352          1119        3471
2   42    2             5344         2950          1529        4479
3   66    2                0            0             0           0
4   20    2             7765         4144          1992        6136
5   58    2                0            0             0           0
6   15    2             1207          286           690         976
7    6    2             7247         3881          1891        5772
8    2    2            21842        20214             0       20214
9   64    3             2286          844          1094        1938
10  14    3              412        

In [45]:
# calculate ratio
def get_ratio(total_num, sum_num):
    if total_num == 0:
        return 0
    else:
        return sum_num / total_num
dc_order['ratio'] = dc_order.apply(lambda x: get_ratio(x['total_order_num'], x['sum_DC_RDC']), axis=1)


In [47]:
dc_order.sort_values(by=['sum_DC_RDC'], ascending=False, inplace=True)
dc_order.to_csv('./pre_data/dc_order.csv', index=False)
print(dc_order)

    DC  RDC  total_order_num  order_by_DC  order_by_RDC  sum_DC_RDC     ratio
23   5    5            29222        27276             0       27276  0.933406
8    2    2            21842        20214             0       20214  0.925465
38   9    9            20637        19069             0       19069  0.924020
20   4    4            18197        17070             0       17070  0.938067
54  24   24            10944        10214             0       10214  0.933297
41  27    9            10759         1842          7078        8920  0.829073
36   7    7             7279         6606             0        6606  0.907542
4   20    2             7765         4144          1992        6136  0.790212
33  31    7             7098         4033          1792        5825  0.820654
7    6    2             7247         3881          1891        5772  0.796468
22  28    4             6374         3309          1897        5206  0.816756
27  26    5             6911         2345          2771        5